https://github.com/tfs4/liar_dataset

In [ ]:
pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve
from sentence_transformers import SentenceTransformer

In [ ]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

train = train.iloc[:,[2,7,1]]
train.columns = ['text','party','label']
valid = valid.iloc[:,[2,7,1]]
valid.columns = ['text','party','label']
test = test.iloc[:,[2,7,1]]
test.columns = ['text','party','label']

def consolidate_label(x):
  a = []
  for i in x['label']:
    if i=='false':
      a.append('pants-fire')
    elif i=='half-true':
      a.append('barely-true')
    elif i=='mostly-true':
      a.append('true')
    else:
      a.append(i)
  return a

train['label'] = consolidate_label(train)
valid['label'] = consolidate_label(valid)
test['label'] = consolidate_label(test)

le = LabelEncoder()
le.fit(['pants-fire','barely-true','true'])
train['label'] = le.transform(train['label'])
valid['label'] = le.transform(valid['label'])
test['label'] = le.transform(test['label'])

print(train.shape)
print(valid.shape)
print(test.shape)

(10269, 3)
(1284, 3)
(1283, 3)


In [ ]:
le.classes_

array(['barely-true', 'pants-fire', 'true'], dtype='<U11')

In [ ]:
test['label'].value_counts()

label
0    481
2    460
1    342
Name: count, dtype: int64

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(train['text'])
embeddings_test = model.encode(test['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
trainX = pd.DataFrame(embeddings)
trainY = train['label']
testX = pd.DataFrame(embeddings_test)
testY = test['label']

In [ ]:
mod = LogisticRegression(penalty='l1', solver='liblinear')
mod.fit(trainX,trainY)

print(roc_auc_score(testY, mod.predict_proba(testX), average='macro', multi_class='ovr'))
print(confusion_matrix(testY, mod.predict(testX)))
print(classification_report(testY,mod.predict(testX), digits=5, target_names=le.classes_))

0.6199003164745932
[[230  91 160]
 [143  99 100]
 [159  60 241]]
              precision    recall  f1-score   support

 barely-true    0.43233   0.47817   0.45410       481
  pants-fire    0.39600   0.28947   0.33446       342
        true    0.48104   0.52391   0.50156       460

    accuracy                        0.44427      1283
   macro avg    0.43646   0.43052   0.43004      1283
weighted avg    0.44011   0.44427   0.43922      1283

